# SQLite and SQLAlchemy

## SQLite

In [7]:
import sqlite3

In [8]:
conn = sqlite3.connect('test2.db') # test2.db파일에 대해서 connection

In [9]:
cur = conn.cursor() # connection에 대해서 cursor 생성

In [10]:
cur.executescript("""
create table Artist(
    id integer not null primary key autoincrement unique,
    name text
    );
create table Genre(
    id integer not null primary key autoincrement unique,
    name text
    );
create table Album(
    id integer not null primary key autoincrement unique,
    title text,
    artist_id integer
    );
create table Track(
    id integer not null primary key autoincrement unique,
    title text,
    length integer,
    rating integer,
    count integer,
    album_id integer,
    genre_id integer
    );
insert into Artist (name) values('Led Zepplin');
insert into Artist (name) values('AC/DC');
insert into Genre (name) values('Rock');
insert into Genre (name) values('Metal');
insert into Album (title, artist_id) values('Who Made Who', 2);
insert into Album (title, artist_id) values('IV',1);
insert into Track (title, rating, length, count, album_id, genre_id) values ('Black Dog', 5, 297, 0, 2, 1);
insert into Track (title, rating, length, count, album_id, genre_id) values ('Stairway', 5, 482, 0, 2, 1);
insert into Track (title, rating, length, count, album_id, genre_id) values ('About to Rock', 5, 313, 0, 1, 2);
insert into Track (title, rating, length, count, album_id, genre_id) values ('Who Made Who', 5, 207, 0, 1, 2);
""")

# executescript를 통해서 여러 테이블 생성 및 엔트리 삽입을 한꺼번에 하고 있다.

In [11]:
cur.execute("select * from artist") # artist 테이블의 모든 엔트리를 불러온다.

In [12]:
print(cur.fetchall()) # 불러온 모든 데이터들을 출력

[(1, 'Led Zepplin'), (2, 'AC/DC')]


In [13]:
artist = cur.fetchall() # 불러온 모든 데이터들은 list 형식으로 반환된다.
type(artist)

list

In [14]:
cur.execute("select * from Track") # Track 테이블의 모든 엔트리들을 불러온다.

In [15]:
print(cur.fetchall()) # 엔트리들을 한꺼번에 출력해준다.

[(1, 'Black Dog', 297, 5, 0, 2, 1), (2, 'Stairway', 482, 5, 0, 2, 1), (3, 'About to Rock', 313, 5, 0, 1, 2), (4, 'Who Made Who', 207, 5, 0, 1, 2)]


In [16]:
cur.execute("select * from artist, album") # artist, album의 모든 엔트리들을 불러옴

### cross join

In [17]:
for row in cur.fetchall(): #우리는 2개를 입력했는데 왜 4개나 나오지?
    print(row) 
#join문으로 명시하지 않고, join을 했기때문에 cross join이 되어 섞여서 나오기 때문이다.

(1, 'Led Zepplin', 1, 'Who Made Who', 2)
(1, 'Led Zepplin', 2, 'IV', 1)
(2, 'AC/DC', 1, 'Who Made Who', 2)
(2, 'AC/DC', 2, 'IV', 1)


In [18]:
cur.execute("select * from artist join album") #cross join ( 모든 경우의 수를 출력)

In [19]:
for row in cur.fetchall(): 
    print(row) #cross join된 결과값을 출력

(1, 'Led Zepplin', 1, 'Who Made Who', 2)
(1, 'Led Zepplin', 2, 'IV', 1)
(2, 'AC/DC', 1, 'Who Made Who', 2)
(2, 'AC/DC', 2, 'IV', 1)


### inner join

In [20]:
cur.execute('select * from artist join album on artist.id = album.artist_id') #inner join

In [21]:
for row in cur.fetchall(): 
    print(row)

(2, 'AC/DC', 1, 'Who Made Who', 2)
(1, 'Led Zepplin', 2, 'IV', 1)


In [22]:
cur.execute('select artist.name, album.title from artist join album on artist.id = album.artist_id')

In [23]:
for row in cur.fetchall(): 
    print(row) #더 깔끔하게 보이기 위해 select에서 출력할 것들만 선택

('AC/DC', 'Who Made Who')
('Led Zepplin', 'IV')


In [24]:
cur.execute("select * from track join album join genre join artist on  album.id = track.album_id and track.genre_id = genre.id and album.artist_id = artist.id")

In [25]:
for row in cur.fetchall(): 
    print(row)
#만약 이것이 보기 힘들다면 select문에서 필요한 column을 명시해주어 프린트한다.

(1, 'Black Dog', 297, 5, 0, 2, 1, 2, 'IV', 1, 1, 'Rock', 1, 'Led Zepplin')
(2, 'Stairway', 482, 5, 0, 2, 1, 2, 'IV', 1, 1, 'Rock', 1, 'Led Zepplin')
(3, 'About to Rock', 313, 5, 0, 1, 2, 1, 'Who Made Who', 2, 2, 'Metal', 2, 'AC/DC')
(4, 'Who Made Who', 207, 5, 0, 1, 2, 1, 'Who Made Who', 2, 2, 'Metal', 2, 'AC/DC')


### dump
#### : 백업 프로그램이 백업 대상에 대해서, SQL 구문의 파일을 만들고, 복원 할 때는 서버에 SQL문을 실행해서, 백업 할 때의 상태로 만드는 기법이다.

In [26]:
with open('test.dump', 'w') as f:
    for line in conn.iterdump():
        f.write("%s\n" % line)
# dump 보내기 

In [27]:
data = None
with open('test.dump', 'r') as f:
    data = f.read() #dump한 파일 읽기. string이기 때문에 메모장에서도 열림..

In [28]:
data

'BEGIN TRANSACTION;\nCREATE TABLE Album(\n    id integer not null primary key autoincrement unique,\n    title text,\n    artist_id integer\n    );\nINSERT INTO "Album" VALUES(1,\'Who Made Who\',2);\nINSERT INTO "Album" VALUES(2,\'IV\',1);\nCREATE TABLE Artist(\n    id integer not null primary key autoincrement unique,\n    name text\n    );\nINSERT INTO "Artist" VALUES(1,\'Led Zepplin\');\nINSERT INTO "Artist" VALUES(2,\'AC/DC\');\nCREATE TABLE Genre(\n    id integer not null primary key autoincrement unique,\n    name text\n    );\nINSERT INTO "Genre" VALUES(1,\'Rock\');\nINSERT INTO "Genre" VALUES(2,\'Metal\');\nCREATE TABLE Track(\n    id integer not null primary key autoincrement unique,\n    title text,\n    length integer,\n    rating integer,\n    count integer,\n    album_id integer,\n    genre_id integer\n    );\nINSERT INTO "Track" VALUES(1,\'Black Dog\',297,5,0,2,1);\nINSERT INTO "Track" VALUES(2,\'Stairway\',482,5,0,2,1);\nINSERT INTO "Track" VALUES(3,\'About to Rock\',313

In [29]:
conn = sqlite3.connect("dump.db")
cur = conn.cursor()
cur.executescript(data) #덤프한 파일을 읽어서 cursor가 실행하도록 했다.
# dump한 부분들이 data에 저장되어 있기 때문에 executescript를 통해서 해당 dump를 실행하면
# 이전에 백업했던 db를 복원할 수 있다.

In [30]:
conn = sqlite3.connect('test2.db')

In [31]:
cur = conn.cursor()

In [32]:
cur.execute("insert into track (title) values ('아무거나')")

In [33]:
cur.execute("select * from track")

In [34]:
print(cur.fetchall())
# 이전에 백업해두었던 부분들도 포함되어 있는 것을 알 수 있다.

[(1, 'Black Dog', 297, 5, 0, 2, 1), (2, 'Stairway', 482, 5, 0, 2, 1), (3, 'About to Rock', 313, 5, 0, 1, 2), (4, 'Who Made Who', 207, 5, 0, 1, 2), (5, '아무거나', None, None, None, None, None)]


### left join(거꾸로 할 경우 right join)

In [35]:
cur.execute('select * from track left join album on album.id = track.album_id')
#SQLite에서는 left join을 지원. but right join을 지원해주지 않는다.

In [36]:
for row in cur.fetchall(): 
    print(row)

(1, 'Black Dog', 297, 5, 0, 2, 1, 2, 'IV', 1)
(2, 'Stairway', 482, 5, 0, 2, 1, 2, 'IV', 1)
(3, 'About to Rock', 313, 5, 0, 1, 2, 1, 'Who Made Who', 2)
(4, 'Who Made Who', 207, 5, 0, 1, 2, 1, 'Who Made Who', 2)
(5, '아무거나', None, None, None, None, None, None, None, None)


In [37]:
cur.execute('select * from album left join track on album.id = track.album_id')
#right join을 지원해주지 않기 때문에 album과 track을 바꾸어서 right join 구현

In [38]:
for row in cur.fetchall(): 
    print(row)

(1, 'Who Made Who', 2, 3, 'About to Rock', 313, 5, 0, 1, 2)
(1, 'Who Made Who', 2, 4, 'Who Made Who', 207, 5, 0, 1, 2)
(2, 'IV', 1, 1, 'Black Dog', 297, 5, 0, 2, 1)
(2, 'IV', 1, 2, 'Stairway', 482, 5, 0, 2, 1)


# SQLAlchemy

#### : SQLAlchemy는 Python SQL toolkit이며 또한 어플 개발자들에게 SQL의 유연성과 power를 제공하는 ORM(Object Relational Mapper)이다.

##### ORM(Object Relational Mapper) : 객체와 RDB간의 mapping을 지원해주는 framework


In [39]:
!pip list # 현재 다운로드되어진 모듈들에 대해서 나열해준다.

Package                            Version           
---------------------------------- ------------------
alabaster                          0.7.10            
anaconda-client                    1.6.5             
anaconda-navigator                 1.6.9             
anaconda-project                   0.8.0             
asn1crypto                         0.22.0            
astroid                            1.5.3             
astropy                            2.0.2             
babel                              2.5.0             
backports.shutil-get-terminal-size 1.0.0             
beautifulsoup4                     4.6.0             
bitarray                           0.8.1             
bkcharts                           0.2               
blaze                              0.11.3            
bleach                             1.5.0             
bokeh                              0.12.10           
boto                               2.48.0            
Bottleneck                  

You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [40]:
import sqlalchemy

In [41]:
sqlalchemy.__version__ # sqlalchemy의 버전

'1.1.13'

In [42]:
from sqlalchemy import *

In [43]:
engine = create_engine("sqlite:///mydb.db", echo=True)
# engine 객체를 생성해주며, DB를 다룰 수 있도록 연결해준다.
# echo flag는 SQLAlchemy의 log를 세팅해주는 flag로서 파이썬의 표준 logging 모듈을 통해 생성된다.


In [47]:
print(engine)

Engine(sqlite:///mydb.db)


In [48]:
from sqlalchemy import MetaData #MetaData 불러옴

In [ ]:
metadata = MetaData() # MetaData객체 생성

여기서 metadata는 여러 table들을 저장하고 있는 객체로서 schema 구조와 연관되어 있다.

In [51]:
print(metadata) #현재 metadata는 연결되어있는것이 없음

MetaData(bind=None)


In [52]:
from sqlalchemy import Table, Column, Integer, String, ForeignKey

users = Table('users', metadata,
             Column('id', Integer, primary_key=True),
             Column('name', String),
             Column('fullname', String),
) # users라는 테이블 객체 생성

In [53]:
addresses = Table('addresses', metadata,
            Column('id', Integer, primary_key = True),
            Column('user_id', None, ForeignKey('users.id')), #'user_id'는 column명, 'users.id'는 실제로 참조하는 것
            Column('email_address', String, nullable=False)
)

metadata.create_all(engine)
# metadata에 저장된 table들을 engine과 연결하여 생성시킨다.

2018-10-08 17:16:05,700 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1


INFO:sqlalchemy.engine.base.Engine:SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1


2018-10-08 17:16:05,711 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


2018-10-08 17:16:05,730 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1


...

INFO:sqlalchemy.engine.base.Engine:
CREATE TABLE addresses (
	id INTEGER NOT NULL, 
	user_id INTEGER, 
	email_address VARCHAR NOT NULL, 
	PRIMARY KEY (id), 
	FOREIGN KEY(user_id) REFERENCES users (id)
)




2018-10-08 17:16:05,826 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


2018-10-08 17:16:05,838 INFO sqlalchemy.engine.base.Engine COMMIT


INFO:sqlalchemy.engine.base.Engine:COMMIT


In [54]:
for row in metadata.tables:
    print(row)

users
addresses


In [55]:
print(metadata) #SQL문 없이도 데이터베이스를 수월하게 다룰 수 있다.

MetaData(bind=None)


In [56]:
users.insert() #리턴하는 것이 object형임.. sql이 아님

In [57]:
print(users.insert()) #이렇게 내부적으로 SQL문을 갖고 있음.

INSERT INTO users (id, name, fullname) VALUES (:id, :name, :fullname)


In [58]:
insert = users.insert().values(name="kim", fullname="Anonymous, Kim") 
#id는 primarykey로 넣었으므로, name과 fullname을 넣는다.

In [59]:
print(users.insert().values(name="kim", fullname="Anonymous, Kim")) 
#insert문이 만들어져있음

INSERT INTO users (name, fullname) VALUES (:name, :fullname)


In [60]:
insert.compile() # 해당 sql문을 compile해준다.

In [61]:
insert.compile().params

{'fullname': 'Anonymous, Kim', 'name': 'kim'}

In [62]:
insert.bind = engine #engine에 bind를 주는 것이다. connection이 필요함
#앞에서 만든 insert객체를 engine에다가 binding해줌.

In [63]:
conn = engine.connect()

In [64]:
result = conn.execute(insert) #엔진에 바인딩된 connection을 통해서 실행하는것임.
#실제 connector를 통해서 SQLite로 넘어간 것임.

2018-10-08 17:27:12,670 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname) VALUES (?, ?)


INFO:sqlalchemy.engine.base.Engine:INSERT INTO users (name, fullname) VALUES (?, ?)


2018-10-08 17:27:12,674 INFO sqlalchemy.engine.base.Engine ('kim', 'Anonymous, Kim')


INFO:sqlalchemy.engine.base.Engine:('kim', 'Anonymous, Kim')


2018-10-08 17:27:12,681 INFO sqlalchemy.engine.base.Engine COMMIT


INFO:sqlalchemy.engine.base.Engine:COMMIT


In [65]:
result.inserted_primary_key

[1]

엔진 하나에서 다 한 것...

In [66]:
insert = users.insert() #이미 어느 테이블에 대해서 작업을 해야하는지 알고 있음.
result = conn.execute(insert, name="lee", fullname = "Unknown, Lee")

2018-10-08 17:30:04,182 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname) VALUES (?, ?)


INFO:sqlalchemy.engine.base.Engine:INSERT INTO users (name, fullname) VALUES (?, ?)


2018-10-08 17:30:04,191 INFO sqlalchemy.engine.base.Engine ('lee', 'Unknown, Lee')


INFO:sqlalchemy.engine.base.Engine:('lee', 'Unknown, Lee')


2018-10-08 17:30:04,202 INFO sqlalchemy.engine.base.Engine COMMIT


INFO:sqlalchemy.engine.base.Engine:COMMIT


In [67]:
result.inserted_primary_key

[2]

In [68]:
conn.execute(addresses.insert(), [
    {"user_id":1, "email_address":"anonymous.kim@test.com"},
    {"user_id":2, "email_address":"unknown.lee@test.com"}
]) #이렇게 여러개를 넣을 경우 dict 타입으로 묶어서 넣어준다.

2018-10-08 17:30:47,009 INFO sqlalchemy.engine.base.Engine INSERT INTO addresses (user_id, email_address) VALUES (?, ?)


INFO:sqlalchemy.engine.base.Engine:INSERT INTO addresses (user_id, email_address) VALUES (?, ?)


2018-10-08 17:30:47,018 INFO sqlalchemy.engine.base.Engine ((1, 'anonymous.kim@test.com'), (2, 'unknown.lee@test.com'))


INFO:sqlalchemy.engine.base.Engine:((1, 'anonymous.kim@test.com'), (2, 'unknown.lee@test.com'))


2018-10-08 17:30:47,031 INFO sqlalchemy.engine.base.Engine COMMIT


INFO:sqlalchemy.engine.base.Engine:COMMIT


In [69]:
from sqlalchemy.sql import select

query = select([users]) #users 테이블 자체를 넣을 때는 []로 싸서 넣어야함.
#다만 column을 넣을 때는 users.column명 으로 넣어주면 된다.

In [70]:
result = conn.execute(query)

2018-10-08 17:31:03,873 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users


INFO:sqlalchemy.engine.base.Engine:SELECT users.id, users.name, users.fullname 
FROM users


2018-10-08 17:31:03,884 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


In [71]:
for row in result:
    print(row)

(1, 'kim', 'Anonymous, Kim')
(2, 'lee', 'Unknown, Lee')


In [72]:
result = conn.execute(select([users.c.name, users.c.fullname]))

for row in result:
    print(row)

2018-10-08 17:31:38,961 INFO sqlalchemy.engine.base.Engine SELECT users.name, users.fullname 
FROM users


INFO:sqlalchemy.engine.base.Engine:SELECT users.name, users.fullname 
FROM users


2018-10-08 17:31:38,970 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


('kim', 'Anonymous, Kim')
('lee', 'Unknown, Lee')


In [73]:
result = conn.execute(query)

row = result.fetchone() #결과값에서 record를 1개씩 가져옴.

2018-10-08 17:32:09,175 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users


INFO:sqlalchemy.engine.base.Engine:SELECT users.id, users.name, users.fullname 
FROM users


2018-10-08 17:32:09,187 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


In [74]:
print(row)

(1, 'kim', 'Anonymous, Kim')


In [75]:
row = result.fetchone()
print(row)

(2, 'lee', 'Unknown, Lee')


In [76]:
row = result.fetchone()
print(row)

None


In [77]:
result = conn.execute(query)

row = result.fetchone() #결과값에서 record를 1개씩 가져옴.

2018-10-08 17:32:40,448 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users


INFO:sqlalchemy.engine.base.Engine:SELECT users.id, users.name, users.fullname 
FROM users


2018-10-08 17:32:40,458 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


In [78]:
result = conn.execute(query)
rows = result.fetchall()

2018-10-08 17:32:55,558 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users


INFO:sqlalchemy.engine.base.Engine:SELECT users.id, users.name, users.fullname 
FROM users


2018-10-08 17:32:55,570 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


In [79]:
for row in rows:
    print(row[0], row[1], row[2])

1 kim Anonymous, Kim
2 lee Unknown, Lee


In [80]:
result = conn.execute(select([users.c.name, users.c.fullname]))

2018-10-08 17:33:14,451 INFO sqlalchemy.engine.base.Engine SELECT users.name, users.fullname 
FROM users


INFO:sqlalchemy.engine.base.Engine:SELECT users.name, users.fullname 
FROM users


2018-10-08 17:33:14,460 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


In [81]:
for row in result:
    print(row)

('kim', 'Anonymous, Kim')
('lee', 'Unknown, Lee')


#####  AND, OR, NOT

In [82]:
from sqlalchemy import and_, or_, not_

print(users.c.id == addresses.c.user_id)

users.id = addresses.user_id


In [83]:
print(or_(users.c.id == addresses.c.user_id,users.c.id == 1))

users.id = addresses.user_id OR users.id = :id_1


In [84]:
print((users.c.id == addresses.c.user_id) | (users.c.id==1))

users.id = addresses.user_id OR users.id = :id_1


SQL문을 모르더라도 파이프라인으로 엮기만 해도 됨

In [85]:
result = conn.execute(select([users]).where(users.c.id==1))

2018-10-08 17:34:05,669 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users 
WHERE users.id = ?


INFO:sqlalchemy.engine.base.Engine:SELECT users.id, users.name, users.fullname 
FROM users 
WHERE users.id = ?


2018-10-08 17:34:05,679 INFO sqlalchemy.engine.base.Engine (1,)


INFO:sqlalchemy.engine.base.Engine:(1,)


In [86]:
result = conn.execute(select([users]).where(users.c.name.like("%i%")))

2018-10-08 17:34:25,323 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users 
WHERE users.name LIKE ?


INFO:sqlalchemy.engine.base.Engine:SELECT users.id, users.name, users.fullname 
FROM users 
WHERE users.name LIKE ?


2018-10-08 17:34:25,334 INFO sqlalchemy.engine.base.Engine ('%i%',)


INFO:sqlalchemy.engine.base.Engine:('%i%',)


In [87]:
for row in result:
    print(row)

(1, 'kim', 'Anonymous, Kim')


In [88]:
result = conn.execute(select([users, addresses]).where(users.c.id == addresses.c.user_id).where(users.c.name.like("k%")))
#where절을 2개 묶은 것이다. .을 통해서 뒤에 묶어 줄 수 있다.

2018-10-08 17:34:56,383 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname, addresses.id, addresses.user_id, addresses.email_address 
FROM users, addresses 
WHERE users.id = addresses.user_id AND users.name LIKE ?


INFO:sqlalchemy.engine.base.Engine:SELECT users.id, users.name, users.fullname, addresses.id, addresses.user_id, addresses.email_address 
FROM users, addresses 
WHERE users.id = addresses.user_id AND users.name LIKE ?


2018-10-08 17:34:56,389 INFO sqlalchemy.engine.base.Engine ('k%',)


INFO:sqlalchemy.engine.base.Engine:('k%',)


In [89]:
for row in result:
    print(row)

(1, 'kim', 'Anonymous, Kim', 1, 1, 'anonymous.kim@test.com')


##### Join

In [90]:
from sqlalchemy import join

In [91]:
print(users.join(addresses))
#이미 PK, FK를 설정해주었기 때문에 명시하지 않고도 자동으로 실행 된다.

users JOIN addresses ON users.id = addresses.user_id


In [92]:
print(users.join(addresses, users.c.id == addresses.c.user_id))
#이렇게 PK, FK를 직접 써줘도 상관은 없다

users JOIN addresses ON users.id = addresses.user_id


SQLite에서는 left join 이외에는 지원하지 않았음.
다만 다른 DBMS를 쓸 때는 필요할 수 있다.

Class 객체상의 디비나, 원래 데이터베이스나 갖고 있는 데이터는 동일

In [93]:
result = conn.execute(select([users]).select_from(users.join(addresses)))
#inner join 되었다.

2018-10-08 17:37:42,439 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users JOIN addresses ON users.id = addresses.user_id


INFO:sqlalchemy.engine.base.Engine:SELECT users.id, users.name, users.fullname 
FROM users JOIN addresses ON users.id = addresses.user_id


2018-10-08 17:37:42,448 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


In [94]:
for row in result:
    print(row)

(1, 'kim', 'Anonymous, Kim')
(2, 'lee', 'Unknown, Lee')


In [95]:
result = conn.execute(select([users.c.name, addresses.c.email_address]).select_from(users.join(addresses)))
#inner join 되었다.

2018-10-08 17:38:51,351 INFO sqlalchemy.engine.base.Engine SELECT users.name, addresses.email_address 
FROM users JOIN addresses ON users.id = addresses.user_id


INFO:sqlalchemy.engine.base.Engine:SELECT users.name, addresses.email_address 
FROM users JOIN addresses ON users.id = addresses.user_id


2018-10-08 17:38:51,361 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


In [96]:
for row in result:
    print(row)

('kim', 'anonymous.kim@test.com')
('lee', 'unknown.lee@test.com')


In [97]:
artist = Table("Artist", metadata, 
               Column("id", Integer, primary_key=True),
               Column("name", String, nullable=False),
               extend_existing=True)

In [98]:
album = Table("Album", metadata, 
              Column("id", Integer, primary_key=True),
              Column("title", String, nullable=False),
              Column("artist_id", Integer, ForeignKey("Artist.id")),
              extend_existing=True)
genre = Table("Genre", metadata, 
              Column("id", Integer, primary_key=True),
              Column("name", String, nullable=False),
              extend_existing=True)

track = Table("Track", metadata,
              Column("id", Integer, primary_key=True),
              Column("title", String, nullable=False),
              Column("length", Integer, nullable=False),
              Column("rating", Integer, nullable=False),
              Column("count", Integer, nullable=False),
              Column("album_id", Integer, ForeignKey("Album.id")),
              Column("genre_id", Integer, ForeignKey("Genre.id")),
              extend_existing=True)

metadata.create_all(engine)

2018-10-08 17:40:51,754 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")


INFO:sqlalchemy.engine.base.Engine:PRAGMA table_info("users")


2018-10-08 17:40:51,763 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


2018-10-08 17:40:51,777 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("addresses")


...

INFO:sqlalchemy.engine.base.Engine:
CREATE TABLE "Track" (
	id INTEGER NOT NULL, 
	title VARCHAR NOT NULL, 
	length INTEGER NOT NULL, 
	rating INTEGER NOT NULL, 
	count INTEGER NOT NULL, 
	album_id INTEGER, 
	genre_id INTEGER, 
	PRIMARY KEY (id), 
	FOREIGN KEY(album_id) REFERENCES "Album" (id), 
	FOREIGN KEY(genre_id) REFERENCES "Genre" (id)
)




2018-10-08 17:40:51,922 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


2018-10-08 17:40:51,938 INFO sqlalchemy.engine.base.Engine COMMIT


INFO:sqlalchemy.engine.base.Engine:COMMIT


In [99]:
artist

Table('Artist', MetaData(bind=None), Column('id', Integer(), table=<Artist>, primary_key=True, nullable=False), Column('name', String(), table=<Artist>, nullable=False), schema=None)

In [100]:
tables = metadata.tables
for table in tables:
    print(table)
    
print("---------------------------------------------")
    
for table in engine.table_names():
    print(table)
    

users
addresses
Artist
Album
Genre
Track
---------------------------------------------
2018-10-08 17:41:39,214 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name


INFO:sqlalchemy.engine.base.Engine:SELECT name FROM sqlite_master WHERE type='table' ORDER BY name


2018-10-08 17:41:39,228 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


Album
Artist
Genre
Track
addresses
users


In [101]:
for row in metadata.tables: #만들어낸 table들
    print(row)

users
addresses
Artist
Album
Genre
Track


In [102]:
users.drop(engine)

2018-10-08 17:42:11,033 INFO sqlalchemy.engine.base.Engine 
DROP TABLE users


INFO:sqlalchemy.engine.base.Engine:
DROP TABLE users


2018-10-08 17:42:11,043 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


2018-10-08 17:42:11,077 INFO sqlalchemy.engine.base.Engine COMMIT


INFO:sqlalchemy.engine.base.Engine:COMMIT


In [103]:
for row in metadata.tables: #만들어낸 table들
    print(row)

users
addresses
Artist
Album
Genre
Track


In [104]:
artist.insert()

In [105]:
conn = engine.connect()

In [106]:
#insert

conn.execute(artist.insert(),[
    {"name":"Led Zepplin"},
    {"name":"AC/DC"}
    
])
conn.execute(album.insert(),[
    {"id":1,"title":"IV","artist_id":1},
    {"id":2,"title":"Who Made Who","artist_id":2}
    
])
conn.execute(genre.insert(),[
    {"name":"Rock"},
    {"name":"Metal"}
])

conn.execute(track.insert(),[
    {"title":"Black Dog","rating":5,"length":297,"count":0,"album_id:":1,"genre_id":1},
    {"title":"StairWay to Heaven","rating":5,"length":482,"count":0,"album_id:":1,"genre_id":1},
    {"title":"About to rock","rating":5,"length":313,"count":0,"album_id:":2,"genre_id":2},
    {"title":"Who Made Who","rating":5,"length":297,"count":0,"album_id:":2,"genre_id":2},
    
])

2018-10-08 17:42:43,171 INFO sqlalchemy.engine.base.Engine INSERT INTO "Artist" (name) VALUES (?)


INFO:sqlalchemy.engine.base.Engine:INSERT INTO "Artist" (name) VALUES (?)


2018-10-08 17:42:43,178 INFO sqlalchemy.engine.base.Engine (('Led Zepplin',), ('AC/DC',))


INFO:sqlalchemy.engine.base.Engine:(('Led Zepplin',), ('AC/DC',))


2018-10-08 17:42:43,190 INFO sqlalchemy.engine.base.Engine COMMIT


...

INFO:sqlalchemy.engine.base.Engine:INSERT INTO "Track" (title, length, rating, count, genre_id) VALUES (?, ?, ?, ?, ?)


2018-10-08 17:42:43,257 INFO sqlalchemy.engine.base.Engine (('Black Dog', 297, 5, 0, 1), ('StairWay to Heaven', 482, 5, 0, 1), ('About to rock', 313, 5, 0, 2), ('Who Made Who', 297, 5, 0, 2))


INFO:sqlalchemy.engine.base.Engine:(('Black Dog', 297, 5, 0, 1), ('StairWay to Heaven', 482, 5, 0, 1), ('About to rock', 313, 5, 0, 2), ('Who Made Who', 297, 5, 0, 2))


2018-10-08 17:42:43,262 INFO sqlalchemy.engine.base.Engine COMMIT


INFO:sqlalchemy.engine.base.Engine:COMMIT


In [107]:
conn.execute(track.update().values(album_id=1).where(track.c.id<3))
conn.execute(track.update().values(album_id=2).where(track.c.id>2))

2018-10-08 17:42:53,059 INFO sqlalchemy.engine.base.Engine UPDATE "Track" SET album_id=? WHERE "Track".id < ?


INFO:sqlalchemy.engine.base.Engine:UPDATE "Track" SET album_id=? WHERE "Track".id < ?


2018-10-08 17:42:53,070 INFO sqlalchemy.engine.base.Engine (1, 3)


INFO:sqlalchemy.engine.base.Engine:(1, 3)


2018-10-08 17:42:53,084 INFO sqlalchemy.engine.base.Engine COMMIT


...

INFO:sqlalchemy.engine.base.Engine:UPDATE "Track" SET album_id=? WHERE "Track".id > ?


2018-10-08 17:42:53,127 INFO sqlalchemy.engine.base.Engine (2, 2)


INFO:sqlalchemy.engine.base.Engine:(2, 2)


2018-10-08 17:42:53,143 INFO sqlalchemy.engine.base.Engine COMMIT


INFO:sqlalchemy.engine.base.Engine:COMMIT


현재의 과정은 주로 back end 개발에 관련한 것이다.

In [108]:
artistResult = conn.execute(artist.select())
for row in artistResult:
    print(row)

2018-10-08 17:43:19,913 INFO sqlalchemy.engine.base.Engine SELECT "Artist".id, "Artist".name 
FROM "Artist"


INFO:sqlalchemy.engine.base.Engine:SELECT "Artist".id, "Artist".name 
FROM "Artist"


2018-10-08 17:43:19,922 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


(1, 'Led Zepplin')
(2, 'AC/DC')


In [109]:
albumResult = conn.execute(album.select())
for row in artistResult:
    print(row)

2018-10-08 17:43:30,773 INFO sqlalchemy.engine.base.Engine SELECT "Album".id, "Album".title, "Album".artist_id 
FROM "Album"


INFO:sqlalchemy.engine.base.Engine:SELECT "Album".id, "Album".title, "Album".artist_id 
FROM "Album"


2018-10-08 17:43:30,783 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


In [110]:
genreResult = conn.execute(genre.select())
for row in genreResult:
    print(row)

2018-10-08 17:43:42,432 INFO sqlalchemy.engine.base.Engine SELECT "Genre".id, "Genre".name 
FROM "Genre"


INFO:sqlalchemy.engine.base.Engine:SELECT "Genre".id, "Genre".name 
FROM "Genre"


2018-10-08 17:43:42,442 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


(1, 'Rock')
(2, 'Metal')


In [111]:
trackResult = conn.execute(track.select())
for row in trackResult:
    print(row)

2018-10-08 17:43:48,777 INFO sqlalchemy.engine.base.Engine SELECT "Track".id, "Track".title, "Track".length, "Track".rating, "Track".count, "Track".album_id, "Track".genre_id 
FROM "Track"


INFO:sqlalchemy.engine.base.Engine:SELECT "Track".id, "Track".title, "Track".length, "Track".rating, "Track".count, "Track".album_id, "Track".genre_id 
FROM "Track"


2018-10-08 17:43:48,788 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


(1, 'Black Dog', 297, 5, 0, 1, 1)
(2, 'StairWay to Heaven', 482, 5, 0, 1, 1)
(3, 'About to rock', 313, 5, 0, 2, 2)
(4, 'Who Made Who', 297, 5, 0, 2, 2)


In [112]:
trackResult = conn.execute(select([track])
                           .where(
                               and_(
                                   track.c.album_id == 1, 
                                   or_(track.c.genre_id == 1, track.c.genre_id == 2)
                               )))

2018-10-08 17:43:54,814 INFO sqlalchemy.engine.base.Engine SELECT "Track".id, "Track".title, "Track".length, "Track".rating, "Track".count, "Track".album_id, "Track".genre_id 
FROM "Track" 
WHERE "Track".album_id = ? AND ("Track".genre_id = ? OR "Track".genre_id = ?)


INFO:sqlalchemy.engine.base.Engine:SELECT "Track".id, "Track".title, "Track".length, "Track".rating, "Track".count, "Track".album_id, "Track".genre_id 
FROM "Track" 
WHERE "Track".album_id = ? AND ("Track".genre_id = ? OR "Track".genre_id = ?)


2018-10-08 17:43:54,823 INFO sqlalchemy.engine.base.Engine (1, 1, 2)


INFO:sqlalchemy.engine.base.Engine:(1, 1, 2)


In [113]:
for row in trackResult:
    print(row)

(1, 'Black Dog', 297, 5, 0, 1, 1)
(2, 'StairWay to Heaven', 482, 5, 0, 1, 1)


In [114]:
from sqlalchemy import update

In [115]:
conn.execute(track.update().values(genre_id=2).where(track.c.id == 2))
conn.execute(track.update().values(genre_id=1).where(track.c.id == 3))

2018-10-08 17:44:00,086 INFO sqlalchemy.engine.base.Engine UPDATE "Track" SET genre_id=? WHERE "Track".id = ?


INFO:sqlalchemy.engine.base.Engine:UPDATE "Track" SET genre_id=? WHERE "Track".id = ?


2018-10-08 17:44:00,096 INFO sqlalchemy.engine.base.Engine (2, 2)


INFO:sqlalchemy.engine.base.Engine:(2, 2)


2018-10-08 17:44:00,109 INFO sqlalchemy.engine.base.Engine COMMIT


...

INFO:sqlalchemy.engine.base.Engine:UPDATE "Track" SET genre_id=? WHERE "Track".id = ?


2018-10-08 17:44:00,148 INFO sqlalchemy.engine.base.Engine (1, 3)


INFO:sqlalchemy.engine.base.Engine:(1, 3)


2018-10-08 17:44:00,158 INFO sqlalchemy.engine.base.Engine COMMIT


INFO:sqlalchemy.engine.base.Engine:COMMIT


In [116]:
#left outer join, inner join 기준이 되는 것이 track이 되도록, 그리고 album을 JOIN

print(track.join(album))

"Track" JOIN "Album" ON "Album".id = "Track".album_id


In [117]:
result = conn.execute(track.select().select_from(track.join(album)))

for row in result:
    print(row)

2018-10-08 17:44:24,262 INFO sqlalchemy.engine.base.Engine SELECT "Track".id, "Track".title, "Track".length, "Track".rating, "Track".count, "Track".album_id, "Track".genre_id 
FROM "Track" JOIN "Album" ON "Album".id = "Track".album_id


INFO:sqlalchemy.engine.base.Engine:SELECT "Track".id, "Track".title, "Track".length, "Track".rating, "Track".count, "Track".album_id, "Track".genre_id 
FROM "Track" JOIN "Album" ON "Album".id = "Track".album_id


2018-10-08 17:44:24,272 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


(1, 'Black Dog', 297, 5, 0, 1, 1)
(2, 'StairWay to Heaven', 482, 5, 0, 1, 2)
(3, 'About to rock', 313, 5, 0, 2, 1)
(4, 'Who Made Who', 297, 5, 0, 2, 2)


In [118]:
result = conn.execute(select([track.c.title, album.c.title]).select_from(track.join(album)))

for row in result:
    print(row)

2018-10-08 17:44:31,003 INFO sqlalchemy.engine.base.Engine SELECT "Track".title, "Album".title 
FROM "Track" JOIN "Album" ON "Album".id = "Track".album_id


INFO:sqlalchemy.engine.base.Engine:SELECT "Track".title, "Album".title 
FROM "Track" JOIN "Album" ON "Album".id = "Track".album_id


2018-10-08 17:44:31,014 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


('Black Dog', 'IV')
('StairWay to Heaven', 'IV')
('About to rock', 'Who Made Who')
('Who Made Who', 'Who Made Who')


In [119]:
print(track.join(album).join(genre))

"Track" JOIN "Album" ON "Album".id = "Track".album_id JOIN "Genre" ON "Genre".id = "Track".genre_id


In [120]:
print(track.join(album).join(genre).join(artist))

"Track" JOIN "Album" ON "Album".id = "Track".album_id JOIN "Genre" ON "Genre".id = "Track".genre_id JOIN "Artist" ON "Artist".id = "Album".artist_id


In [121]:
result = conn.execute(select([track.c.title, album.c.title, genre.c.name, artist.c.name])
                     .select_from(track.join(album).join(genre).join(artist)))

for row in result:
    print(row)

2018-10-08 17:44:51,370 INFO sqlalchemy.engine.base.Engine SELECT "Track".title, "Album".title, "Genre".name, "Artist".name 
FROM "Track" JOIN "Album" ON "Album".id = "Track".album_id JOIN "Genre" ON "Genre".id = "Track".genre_id JOIN "Artist" ON "Artist".id = "Album".artist_id


INFO:sqlalchemy.engine.base.Engine:SELECT "Track".title, "Album".title, "Genre".name, "Artist".name 
FROM "Track" JOIN "Album" ON "Album".id = "Track".album_id JOIN "Genre" ON "Genre".id = "Track".genre_id JOIN "Artist" ON "Artist".id = "Album".artist_id


2018-10-08 17:44:51,374 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


('Black Dog', 'IV', 'Rock', 'Led Zepplin')
('StairWay to Heaven', 'IV', 'Metal', 'Led Zepplin')
('About to rock', 'Who Made Who', 'Rock', 'AC/DC')
('Who Made Who', 'Who Made Who', 'Metal', 'AC/DC')
